In [32]:
import numpy as np
import pandas as pd
import geopandas as gpd
import warnings
from shapely.geometry import Polygon
import sys
import os
import osmnx as ox
city_geo = ox.geocoder.geocode_to_gdf

In [33]:
thresholds = [300, 600, 1000]

In [34]:
ls_in = ['Abidjan','Addis Ababa','Baghdad','Bangalore','Belgrade','Belo Horizonte','Bogota','Bologna','Cali','Canberra',\
         'Casablanca','Chandigarh','Chelyabinsk','Chengdu','Christchurch','Cochabamba','Copenhagen','Curitiba','Damascus',\
         'Detroit','Dhaka','Dongguan','Fortaleza','Ghent','Guatemala City','Gwangju','Hanoi','Indore','Johannesburg',\
         'Kampala','Karlsruhe','Kathmandu','Kharkiv','Kuala Lumpur','Kumamoto','Lagos','Leicester','Liverpool',\
         'Lubumbashi','Lviv','Manila','Memphis','Mexico City','Mombasa','Nanjing','New Delhi','Oran','Oslo','Perth',\
         'Philadelphia','Pune','Rennes','Rosario','Rostov','San Antonio','San Diego','Santo Domingo','Seattle','Seoul',\
         'Sevilla','Shijiazhuang','Shiraz','Skopje','Surabaya','Tallinn','Tashkent','Tel Aviv','Toronto','Turin',\
         'Utrecht','Utsunomiya','Washington DC','Yaounde','Yerevan','Zibo']

In [35]:
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(ls_in)]
cities_adj = cities_adj.sort_values('City').reset_index()

ls_adj = ['Canberra','Karlsruhe','Oslo']

cities_adj2 = cities[cities['City'].isin(ls_adj)]
cities_adj2 = cities_adj2.sort_values('City').reset_index()

In [36]:
ser = gpd.GeoSeries(crs = 4326)
for i in cities_adj['OSM_area']:
    ser = pd.concat([ser, city_geo(i.split(', ')).dissolve().geometry.centroid])
ser.index = ls_in
ser

Abidjan            POINT (-4.00962 5.34699)
Addis Ababa        POINT (38.78696 8.98080)
Baghdad           POINT (44.37208 33.32650)
Bangalore         POINT (77.59610 12.95079)
Belgrade          POINT (20.42554 44.80930)
                            ...            
Utsunomiya       POINT (139.88657 36.59688)
Washington DC    POINT (-77.01637 38.90483)
Yaounde            POINT (11.50038 3.85368)
Yerevan           POINT (44.50722 40.16141)
Zibo             POINT (118.06205 36.80911)
Length: 75, dtype: geometry

In [37]:
ser.to_file('D:/Dumps/GEE-WP Scores/cities.gpkg')

In [ ]:
%%time
warnings.filterwarnings('ignore')

path = "D:/Dumps/GEE-WP Scores/Gravity/100m grids/Grid_geoms/"
ls_file = os.listdir(path)
ls_file = np.array(ls_file)[[i.rsplit('_')[1] == 'grav3' for i in ls_file]]
ls_file = ls_file[[any([j in i for j in ls_in]) for i in ls_file]]

ls_2 = ls_file[[any([j in i for j in ls_adj]) for i in ls_file]]

warnings.filterwarnings('ignore')
area = [city_geo(i.split(', ')).dissolve().geometry for i in cities_adj2['OSM_ineq_adj']]
files = [gpd.read_file(path+i) for i in ls_2]
scores = [files[i].overlay(gpd.GeoDataFrame(geometry = area[i], crs = 4326)) for i in range(len(files))]

print('preorocessing done')
print('')

cities_scores = list()
for i in enumerate(ls_file):
    city_score = gpd.read_file(path+i[1])
    
    k = 0
    if i[1] in ls_adj:
        city_score2 = scores[k]
        k = k+1
    else:
        city_score2 = city_score
    
    scores_df = city_score.iloc[range(0,len(city_score2),3)]['geometry'].reset_index(drop = True)
    for j in thresholds:
        ls = list()
        score = city_score['tr_'+str(j)].dropna().reset_index(drop =True)
        for k in score:
            if k == 0: 
                ls.append('0 no')
            elif k < j: # one perfect score (no route cost to UGS); score is threshold - route (and entry) cost
                ls.append('1 low')
            elif k < j*2: # two perfect scores
                ls.append('2 mediocre')
            elif k < j*3: # three perfect scores
                ls.append('3 sufficient')
            elif k < j*4: # four perfect scores
                ls.append('4 good')
            else: # above four perfect scores (or 8 UGS on half the threshold distance)
                ls.append('5 excellent')
        print(i[1].rsplit('_')[2].rsplit('.')[0], j)
        scores_df = pd.concat([scores_df, 
                               pd.Series(round(score, 1), name = 'Grav-Sc '+str(j)), 
                               pd.Series(ls, name = 'GravQnt '+str(j))], 
                              axis = 1)
    cities_scores.append(scores_df)
cities_scores[0]

preorocessing done

Abidjan 300
Abidjan 600
Abidjan 1000
Addis Ababa 300
Addis Ababa 600
Addis Ababa 1000
Baghdad 300
Baghdad 600
Baghdad 1000
Bangalore 300
Bangalore 600
Bangalore 1000
Belgrade 300
Belgrade 600
Belgrade 1000
Belo Horizonte 300
Belo Horizonte 600
Belo Horizonte 1000
Bogota 300
Bogota 600
Bogota 1000
Bologna 300
Bologna 600
Bologna 1000
Cali 300
Cali 600
Cali 1000
Canberra 300
Canberra 600
Canberra 1000
Casablanca 300
Casablanca 600
Casablanca 1000
Chandigarh 300
Chandigarh 600
Chandigarh 1000
Chelyabinsk 300
Chelyabinsk 600
Chelyabinsk 1000
Chengdu 300
Chengdu 600
Chengdu 1000
Christchurch 300
Christchurch 600
Christchurch 1000
Cochabamba 300
Cochabamba 600
Cochabamba 1000
Copenhagen 300
Copenhagen 600
Copenhagen 1000
Curitiba 300
Curitiba 600
Curitiba 1000
Damascus 300
Damascus 600
Damascus 1000
Detroit 300
Detroit 600
Detroit 1000
Dhaka 300
Dhaka 600
Dhaka 1000
Dongguan 300
Dongguan 600
Dongguan 1000
Fortaleza 300
Fortaleza 600
Fortaleza 1000
Ghent 300
Ghent 600
Ghen

In [ ]:
%%time
E2SCFA_class = list()
path = "D:/Dumps/GEE-WP Scores/E2SFCA/100m grids/grid_geoms/"
ls_file = np.array(os.listdir(path))
#ls_file = pd.Series(ls_file)[pd.Series(ls_file).str.contains('gpkg')].tolist()
ls_file = ls_file[[any([j in i for j in ls_in]) for i in ls_file]]

ls_2 = ls_file[[any([j in i for j in ls_adj]) for i in ls_file]]

warnings.filterwarnings('ignore')
area = [city_geo(i.split(', ')).dissolve().geometry for i in cities_adj2['OSM_ineq_adj']]
files = [gpd.read_file(path+i) for i in ls_2]
scores = [files[i].overlay(gpd.GeoDataFrame(geometry = area[i], crs = 4326)) for i in range(len(files))]

for i in ls_file:
    city_score = gpd.read_file(path+i)
    
    k = 0
    if i[1] in ls_adj:
        city_score2 = scores[k]
        k = k+1
    else:
        city_score2 = city_score
    
    score_df = city_score[['population']]
    for j in thresholds:
        ls = list()
        score = city_score['Sc-norm '+str(j)]
        for k in score:
            if k == 0: # m2 of UGS per person and weighted with Gaussian (threshold capped) distance decay
                ls.append('0 no')
            elif k < 9: # m2 per person required by the WHO
                ls.append('1 low')
            elif k < 18: # m2 per person twice required by the WHO
                ls.append('2 mediocre')
            elif k < 30: # m2 per person preferred by the WHO
                ls.append('3 sufficient')
            elif k < 100: # over 100m2 per person, the additive value of m2 per person becomes almost obsolete
                ls.append('4 good') # below the 100m2 is still a good score
            else:
                ls.append('5 excellent') # above it will be excellent
        print(i.rsplit('.')[0], j)
        score_df = pd.concat([score_df, 
                              round(pd.Series(score, name = 'E2SCFA-Sc '+str(j)),2), 
                              pd.Series(ls, name = 'E2SCFA-class '+str(j))], 
                             axis = 1)
    E2SCFA_class.append(score_df)
E2SCFA_class[0]

In [ ]:
%%time
scores = list()
write_path = 'D:/Dumps/GEE-WP Scores/city_summary/'
for i in range(len(E2SCFA_class)):
    score = pd.concat([E2SCFA_class[i], cities_scores[i]], axis = 1)
    
    score = score[['geometry','population',
                   'E2SCFA-class 300','E2SCFA-class 600','E2SCFA-class 1000',
                   'GravQnt 300','GravQnt 600','GravQnt 1000',
                   'E2SCFA-Sc 300','E2SCFA-Sc 600','E2SCFA-Sc 1000',
                   'Grav-Sc 300','Grav-Sc 600','Grav-Sc 1000']]
        
    # Adding categories without grid geoms attached to ensure that each category is in a plotted map
    ls2 = pd.DataFrame(list([
                [Polygon(),'5 excellent','5 excellent','5 excellent','5 excellent','5 excellent','5 excellent'],
                [Polygon(),'4 good','4 good','4 good','4 good','4 good','4 good'],
                [Polygon(),'3 sufficient','3 sufficient','3 sufficient','3 sufficient','3 sufficient','3 sufficient'],
                [Polygon(),'2 mediocre','2 mediocre','2 mediocre','2 mediocre','2 mediocre','2 mediocre'],
                [Polygon(),'1 low','1 low','1 low','1 low','1 low','1 low'],
                [Polygon(),'0 no','0 no','0 no','0 no','0 no','0 no']]))
    
    ls2.columns = ['geometry',
                   'E2SCFA-class 300','E2SCFA-class 600','E2SCFA-class 1000',
                   'GravQnt 300','GravQnt 600','GravQnt 1000']
    
    score = pd.concat([score, pd.DataFrame(ls2)])
    
    score = gpd.GeoDataFrame(score, geometry = 'geometry', crs = 4326)
    score.to_file(write_path+cities_adj['City'][i]+'.gpkg')
    score.iloc[:,score.columns != 'geometry'].to_csv(write_path+cities_adj['City'][i]+'.csv')
    print(cities_adj['City'][i])
    scores.append(score)

In [ ]:
[[print(city[1], 
        scores[cities_adj['City'][0]].groupby('E2SCFA-class '+str(t))['E2SCFA-class '+str(t)].count()) 
  for t in thresholds] 
 for city in enumerate(cities)]
sys.stdout.flush()

In [ ]:
summ_df = pd.DataFrame([i.groupby('E2SCFA-class 300')['population'].sum() / i['population'].sum() for i in scores])
summ_df.index = cities_adj['City']
summ_df.to_csv(write_path+'E2SFCA_300_popweighaccess.csv')

In [ ]:
summ_df = pd.DataFrame([i.groupby('GravQnt 300')['population'].sum() / i['population'].sum() for i in scores])
summ_df.index = cities_adj['City']
summ_df.to_csv(write_path+'Gravity_300_popweighaccess.csv')

In [ ]:
summ_df.sort_values('5 excellent')